# 第5章: 大規模言語モデル

## 40. Zero-Shot推論

In [1]:
# Geminiのライブラリのインストール
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.5 MB/s eta 0:00:00


In [2]:
# APIキーの読み込み
from google.colab import userdata
import google.generativeai as genai

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [3]:
import pandas as pd

model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

prompt = df.loc[df["id"] == 401, "question"].values[0]
response = model.generate_content(prompt)
print(response.text)

# Gemini_APIの使用  https://qiita.com/zukki2/items/10bfeb1c4330aa18ff87

正しい年代順は、イ→ウ→アです。

* **イ 嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。**　嵯峨天皇の治世は809年から823年。藤原冬嗣は嵯峨天皇の時代に活躍した人物です。

* **ウ 藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。** 承和の変は842年。良房はこの変の後、権力を掌握し北家の優位を確立しました。

* **ア 藤原時平は，策謀を用いて菅原道真を政界から追放した。** 菅原道真の左遷は894年。藤原時平は、この事件の中心人物でした。


よって、イ、ウ、ア の順になります。



## 41. Few-Shot推論

In [4]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
few_shot_df = df[(df["id"] >= 410) & (df["id"] <= 414)]

few_shot = ""
for _, row in few_shot_df.iterrows():
  few_shot += f"question:{row['question']} answer:{row['answer']} "
question = f"question:{df.loc[df['id'] == 401, 'question'].values[0]} answer:"
prompt = few_shot + question

response = model.generate_content(prompt)
print(response.text)

解答: イ→ウ→ア



## 42. 多肢選択問題の正解率

In [5]:
import time

model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/business_ethics.csv", encoding="utf-8-sig")
correct = df["answer"].tolist()
answer = []

for _, row in df.iterrows():
  prompt = (
    f"question:{row['question']}A:{row['A']}B:{row['B']}C:{row['C']}D:{row['D']}"
    "A, B, C, Dの記号を選択して、いずれかの記号のみを出力してください。記号でない出力は扱えません。"
  )
  while(1):
    response = model.generate_content(prompt)
    resp_text = response.text.strip()
    if (resp_text in ["A", "B", "C", "D"]):
      answer.append(resp_text)
      break
    else:
      time.sleep(4)
      print(response.text)
  time.sleep(4) # 1分に15回のリクエスト制限

df["prediction"] = answer
accuracy = (df["prediction"] == df["answer"]).mean()
print(f"{accuracy * 100:.2f}%")

# sleep関数 https://qiita.com/shi_ei/items/2a88ba1796a4f8f7ad0a

75.29%


## 43. 応答のバイアス

In [6]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/business_ethics.csv", encoding="utf-8-sig")
correct = df["answer"].tolist()
answer = []

for _, row in df.iterrows():
  # プロンプトの一部をエストニア語（母語話者が少ない）に変更
  prompt = (
    f"küsimus:{row['question']}A:{row['A']}B:{row['B']}C:{row['C']}D:{row['D']}"
    "Valige sümbol A, B, C või D ja väljundige ainult üks sümbol.Mitte-sümbolite väljundit ei saa käsitleda."
  )
  while(1):
    response = model.generate_content(prompt)
    resp_text = response.text.strip()
    if (resp_text in ["A", "B", "C", "D"]):
      answer.append(resp_text)
      break
    else:
      time.sleep(4)
  time.sleep(4) # 1分に15回のリクエスト制限

df["prediction"] = answer
accuracy = (df["prediction"] == df["answer"]).mean()
print(f"{accuracy * 100:.2f}%")

71.76%


## 44. 対話

In [7]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

prompt = df.loc[df["id"] == 441, "question"].values[0]
chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

つばめちゃんが間違って乗った急行は、自由が丘から大井町方面へ向かっています。東急大井町線の急行は、自由が丘の次は九品仏駅に停車します。そこで降りて反対方向の電車で一駅戻ると、**自由が丘駅**が目的地になります。



## 45. マルチターン対話

In [8]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

prompt1 = df.loc[df["id"] == 441, "question"].values[0]
prompt2 = df.loc[df["id"] == 451, "question"].values[0]

chat = model.start_chat(history=[])
response = chat.send_message(prompt1)
response = chat.send_message(prompt2)
print(response.text)

東急大井町線で自由が丘駅から大井町方面と反対方向（渋谷方面）の急行に乗車した場合、次の急行停車駅は中目黒駅です。そこで降りて、反対方向（大井町方面）の各駅停車に乗車します。

中目黒駅から各駅停車で大井町方面に向かう場合、自由が丘駅までは以下の駅を通過します。

* 中目黒
* 祐天寺
* 学芸大学
* 都立大学
* 自由が丘

したがって、自由が丘駅まで **5駅** 乗車する必要があります。



## 46. 川柳の生成

In [10]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
Senryu = []

prompt = df.loc[df["id"] == 461, "question"].values[0]
for i in range(10):
  chat = model.start_chat(history=[])
  response = chat.send_message(prompt)
  Senryu.append(response.text.strip())
  time.sleep(4)

for i in range(10):
  print(Senryu[i])

言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き夢
言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き森
言葉解き　未来拓く　深き森


## 47. LLMによる評価

In [11]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
eval = []

question = df.loc[df["id"] == 471, "question"].values[0]
for i in range(10):
  prompt = question + Senryu[i]
  response = model.generate_content(prompt)
  eval.append(f"{Senryu[i]}: {response.text.strip()}")
  time.sleep(4)

for i in range(10):
  print(eval[i])

言葉解き　未来拓く　深き森: 5/10点
言葉解き　未来拓く　深き森: 5/10点
言葉解き　未来拓く　深き森: 5/10点
言葉解き　未来拓く　深き森: 6/10
言葉解き　未来拓く　深き森: 6/10点
言葉解き　未来拓く　深き森: 6/10点
言葉解き　未来拓く　深き夢: 6/10点
言葉解き　未来拓く　深き森: 5/10点
言葉解き　未来拓く　深き森: 6/10点
言葉解き　未来拓く　深き森: 5/10点


## 48. LLMによる評価の頑健性

In [12]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
eval = []

question = df.loc[df["id"] == 481, "question"].values[0]
for i in range(10):
  prompt = question + Senryu[i]
  response = model.generate_content(prompt)
  eval.append(f"{Senryu[i]}: {response.text.strip()}")
  time.sleep(4)

for i in range(10):
  print(eval[i])

言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き夢: 10/10
言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き森: 10/10
言葉解き　未来拓く　深き森: 10/10


## 49. トークン化

In [13]:
import pandas as pd

model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

text = df.loc[df["id"] == 491, "question"].values[0]
question = df.loc[df["id"] == 492, "question"].values[0]
prompt = (f"文章:{text} 質問:{question}")

response = model.generate_content(prompt)
print(response.text)

この文章のトークン数は、正確なカウント方法によって多少異なりますが、概ね**80～90トークン**程度です。  句読点をトークンと数えるか、単語をトークンと数えるか、固有名詞をどう扱うかなどによって変動します。  機械翻訳システムなどによってカウントすると、さらに数値が変動する可能性があります。

